In [1]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
from dqn.policy import PolicyEpsilongGreedy
from dqn.dqn_base import DeepQLearning
from dqn.neural_net import NNLunarLander
from dqn.environment import DQNEnvironment

In [2]:
device = "cpu"
hidden_size = 128

In [3]:
env = DQNEnvironment("LunarLander-v2")

In [4]:
algo = DeepQLearning(env=env.env, policy=PolicyEpsilongGreedy(device), q_net=NNLunarLander(hidden_size, env.observation_size(), env.number_of_actions()))

/Users/meinczinger/.pyenv/versions/3.10.9/envs/rl/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'q_net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['q_net'])`.
  rank_zero_warn(
/Users/meinczinger/.pyenv/versions/3.10.9/envs/rl/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:249: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [5]:
match(device):
    case 'cpu':
        trainer = Trainer(
            max_epochs=10_000,
            callbacks=[EarlyStopping(monitor="episode/Return", mode="max", patience=500)],
        )
    case 'mps':
        trainer = Trainer(
            max_epochs=10_000,
            callbacks=[EarlyStopping(monitor="episode/Return", mode="max", patience=500)],
            accelerator="mps", devices=1
        )


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/meinczinger/.pyenv/versions/3.10.9/envs/rl/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:200: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [6]:
import tempfile
import shutil
shutil.rmtree(tempfile.gettempdir()+"/.tensorboard-info", ignore_errors=True)

%load_ext tensorboard
%tensorboard --logdir lightning_logs --port 6007

In [7]:
trainer.fit(algo)


  | Name         | Type          | Params
-----------------------------------------------
0 | q_net        | NNLunarLander | 18.2 K
1 | target_q_net | NNLunarLander | 18.2 K
-----------------------------------------------
36.4 K    Trainable params
0         Non-trainable params
36.4 K    Total params
0.145     Total estimated model params size (MB)
/Users/meinczinger/.pyenv/versions/3.10.9/envs/rl/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

In [8]:
from tensorboard import notebook
# notebook.list()
notebook.display(port=6007, height=1000)

Selecting TensorBoard with logdir lightning_logs (started 0:05:31 ago; port 6007, pid 6583).


![DQN](dqn_average_return.png)